In [1]:
import gym
import custom_gym
import numpy as np
import torch

env = gym.make('CustomPendulum-v0')
s_dim = env.reset().shape[0]
a_dim = env.action_space.sample().shape[0]
print(s_dim,a_dim)

3 1


C:\Users\toru.hishinuma\anaconda3\envs\note\lib\site-packages\gym\spaces\box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [2]:
offline_data = np.load('np_offline_data.npy')
offline_data = torch.from_numpy(offline_data.astype(np.float32))[:,:,:-1]
print(offline_data.shape)

torch.Size([100, 200, 7])


In [3]:
def datapoint2sads(datapoint):
    s = datapoint[: s_dim]
    a = datapoint[s_dim : (s_dim+a_dim)]
    #ds = datapoint[(s_dim+a_dim) : (2*s_dim+a_dim)] - s
    ds = datapoint[(s_dim+a_dim) : (2*s_dim+a_dim)]
    return s,a,ds



In [4]:
x_dim = s_dim+a_dim
y_dim = s_dim
z_dim = 1
g_dim = 1
#f_phi


In [5]:
from unweighted_vi import unweightedVI
    
vi = unweightedVI(s_dim, a_dim, z_dim, g_dim)

optimizer = torch.optim.Adam(vi.parameters(),lr=0.001)

for i in range(10):
    m = np.random.randint(offline_data.shape[0])
    optimizer.zero_grad()
    loss = vi.loss(offline_data[m,:,:])
    loss.backward()
    optimizer.step()

        #m = np.random.randint(offline_data.shape[0])
        #z_mu_logvar = enc(offline_data[m,:,:-1])
        

